# Normal Estimation
## Abstract
- 本章では、点群の法線推定について説明する。

## Introduction
点群では法線情報を使う場合があり、その様な場面として以下の例が挙げられる。
1. PFH等のHandcrafted featureを生成するため、点ごとの法線情報が必要となる。
2. スキャンした点群から表面情報を持つ3Dオブジェクトデータを作成するため、法線情報を使用する。

しかしながら、法線を取得したい点群自身が座標値のみしか持ってない場合も考えられる。この様に点群が法線情報を持っていない状況には、点群の座標値を参照して法線を推定する方法が解決策になる可能性がある。
本sectionでは、法線情報を持たない点群が与えられた場合に、その法線を推定する方法について紹介する。

本チュートリアルの説明の流れは以下の通りである。
- (1) 座標値XYZのみを持つ点群に対して法線を推定する。内容は以下のsubsectionごとにまとめられる。
  - Estimation with Covariance Matrix
- (2) single viewから取得した座標値のみを持つ点群の法線を推定する。内容は以下のsubsectionごとにまとめられる。
  - Estimation with Covariance Matrix and Single View


In [1]:
%load_ext autoreload
%autoreload 2

## Estimation with Covariance Matrix 

このsubsectionでは、Covariance Matrixを用いたシンプルな手法で、座標値XYZのみを持つ点群に対して点ごとの法線を推定する。この方法では、ある点の近傍点がどの様に分布しているか計算し、点が広がっていない方向を法線として推定する。これは、点群がオブジェクト表面からサンプリングされており、点の分布形状を表面とみなすことができるためである。この点の分布を把握するため、Covariance Matrixを利用する。

本subsectionで利用するモジュールは以下の通り:

In [2]:
# for normal estimation
import numpy as np
from tutlibs.normal_estimation import normal_estimation, normal_orientation

# for description
from tutlibs.io import Points as io
from tutlibs.transformation import Transformation as tr
from tutlibs.visualization import JupyterVisualizer as jv
import inspect

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


法線推定のコードは以下の通り:

In [11]:
# Load a point cloud
xyz, _, data = io.read('../data/bunny_pc.ply')

# Estimate normals
k = 15
estimation_normals = normal_estimation(xyz, k=k)

# Get GT normals
gt_normals = np.stack([data['nx'], data['ny'], data['nz']], axis=-1)

# Visualizaiton
obj_gt_points = jv.point(
    tr.translation(xyz, np.array([1, 0, 0])), # translation for comparing
    gt_normals,
    color_range=[-1, 1],
    point_size=0.02
)
obj_estimation_points = jv.point(
    xyz,
    estimation_normals,
    color_range=[-1, 1],
    point_size=0.02
)
jv.display([obj_gt_points, obj_estimation_points])


Output()

上記の推定では、x軸方向に0~1の間にある点群が法線推定を施した点群(法線推定点群)、1~2の間にある点群がMeshから直接取得した法線を持つ点群(GT法線点群)である。法線は法線マップと呼ばれる色合いによって示されている。
この実装と結果については以下の２つに分けて説明する。

1. Implementation: 法線推定を行う関数の中身について説明
2. Analysis: GT法線点群と違い、法線マップがまだらである理由とその対処


### Implementation
上記コードでは、`normal_estimation`関数に法線推定ほ実装している。この関数の中身は以下の通り:


In [12]:
print(inspect.getsource(normal_estimation))

def normal_estimation(coords:np.ndarray, k:int=10) -> np.ndarray:
    """Estimate normals each point with eigenvector of covariance matrix.
    This function use k nearest neighbor (kNN) to get covariance matrixs.

    Args:
        coords: coordinates of points, (N, 3)
        k: number of neighbor for kNN

    Returns
        normals: estimated normals (N, 3)
    """
    # Get neighbor points. (TODO: add radius)
    idxs, _  = k_nearest_neighbors(coords, coords, k)
    knn_points = gather(coords, idxs)

    # Get covariance matrix of each point.
    knn_mean_points = np.mean(knn_points, axis=-2, keepdims=True)
    deviation = knn_points - knn_mean_points
    deviation_t = deviation.transpose(0, 2, 1)
    covariance_matrixs = np.matmul(deviation_t, deviation) / k # (N, 3, 3)

    # Get eigenvector and eigenvalue of each point
    w, v = np.linalg.eig(covariance_matrixs.transpose(0, 2, 1))
    # w, v = np.linalg.eig(covariance_matrixs)
    w_min_idxs = np.argmin(w, axis=1)

    # Get n

上記実装をコメントで各段分けて説明する。

上記実装で解く問題の定義は次の通り: 点$p=[x, y, z]$を$M$個持つ点群$P=[p_1, p_2, ..., p_m, ..., p_M]$がある時、点群の各点の法線ベクトル$n$を推定する。推定するすべての点の法線を$N=[n_1, n_2, ..., n_M]$とする。

プロセスは次の通り:

1. `Get neighbor points.`: 法線を最近傍手法(本subsectionではkNN)を用いて、点群中の各点$p_m$は近傍点$p'_m=[p'_{m_1}, p'_{m_2}, ..., p'_{m_K}]$($p_m$を含む)を取得する。点群中の全ての点の近傍点は$P'=[p'_1, p'_2, ..., p'_m, ..., p'_M]$となる。
2. `Get covariance matrix of each point.`: $p_m$ごとに$p'_m$の点の分布を取得するため、Covariance Matrixを求める。$\overline{p}$を$p'_m$の平均座標値としたとき、Convariance Matrix $c_m$の式は次の通り:
    $$
    \mathcal{C_m}=\frac{1}{K} \sum_{i=1}^{K} \left(\boldsymbol{p}'_{m_i}-\overline{\boldsymbol{p}}\right) \cdot\left(\boldsymbol{p}'_{m_i}-\overline{\boldsymbol{p}}\right)^{T}
    $$
    $c$はM個あるため、$c_1, c_2, ..., c_m, ..., c_M$を取得する。
3. `Get eigenvector and eigenvalue of each point`: 点が最も広がっていない方向を算出するため、$c_m$から固有値と固有ベクトルを取得する。最も低い固有値は点が最も広がっていない方向を指す固有ベクトルに対応する。
4. `Get normal of each point`: 最も低い固有値に対応する固有ベクトルを法線として、法線を割り振る。


### Analysis
推定された法線の中には、本来の法線の向きとは真逆のものがある。これは、Convariance Matrixを使用して算出された法線の符号を解く術がないからである。この問題に対処する方法の一つとして最小全域木の木構造を用いた方向付が挙げられる。方向付けは以下のコードの通り。


In [14]:
# Load a point cloud
xyz, _, data = io.read('../data/bunny_pc.ply')

# Estimate normals
k = 15
estimation_normals = normal_estimation(xyz, k=k)

# with Orientation
ori_estimation_normals = normal_orientation(xyz, estimation_normals)

# Get GT normals
gt_normals = np.stack([data['nx'], data['ny'], data['nz']], axis=-1)

# visualization
obj_gt_points = jv.point(
    tr.translation(xyz, np.array([2, 0, 0])), # translation for comparing
    gt_normals,
    color_range=[-1, 1],
    point_size=0.02
)
obj_estimation_points = jv.point(
    xyz,
    estimation_normals,
    color_range=[-1, 1],
    point_size=0.02
)
obj_ori_estimation_points = jv.point(
    tr.translation(xyz, np.array([1, 0, 0])), # translation for comparing
    ori_estimation_normals,
    color_range=[-1, 1],
    point_size=0.02
)
jv.display([obj_gt_points, obj_estimation_points, obj_ori_estimation_points])

Output()

出力はx=0~1の点群が法線推定点群、1~2が法線推定点群に方向付を適用したもの、2~3がGT法線点群である。方向付け方法は`normal_orientation`関数によって行われる。


## Estimation with Covariance Matrix and Single View
更新予定


## References